# Advanced Swarmpy

<a href="https://www.python.org/downloads/release/python-3100/" 
target="_blank"><img src="https://img.shields.io/badge/python-3.10-blue.svg" alt="Python Version" /></a>

___

Combining pipeline, masks and ant parameter randomization

The goal here is to warmstart the pheromone trails by applying ACO to a partition of the graph. Therefore we use intermediate pipelines with mask and eventually apply iterations on the entire problem

In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
os.chdir('..')
from copy import deepcopy

from swarmpy_tsp import *
import plotly.express as px


[SwarmPy] NumExpr defaulting to 8 threads.


In [2]:
G1, opt_score = Antcoder('test_set/berlin52')

In [3]:
mask = np.ones_like(G1['e'], dtype=bool)
mid = int(G1['e'].shape[0]/2)
mask[:mid, :mid] = 0

In [4]:
scores=[]
for _ in range(10):
    G = deepcopy(G1)

    aco_with_Daemon1 = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0], ant_params={'mask':mask})),
            ("Sol", SolutionConstructor()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.4,.6])),
    ], iter_max=10, as_step=True, verbose=30
    )

    aco_with_Daemon2 = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0], ant_params={'mask':~mask})),
            ("Sol", SolutionConstructor()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.4,.6]))
            
    ], iter_max=10, as_step=True, verbose=30
    )

    aco_final = ACO_Pipeline(
        [
            ("Planner", RandomizedPlanner(alpha_bounds=[1.0,1.0], beta_bounds=[1.0,6.0])),
            ("Sol", SolutionConstructor()),
            ("DA", DaemonActions()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.1,1])),
            
    ], iter_max=30, last_step=True, verbose=30
    )

    optimus_prime_pipeline = ACO_Pipeline([('a', aco_with_Daemon1), ('b',aco_with_Daemon2), ('d',aco_final)], iter_max=1, metapipeline=True, verbose=30)

    solutions_w_daemon = optimus_prime_pipeline.run(G=G,)
    scores_w_daemon = np.array([el[1] for el in solutions_w_daemon['solutions']]) - opt_score

    scores.append(scores_w_daemon)

SwarmPy |Step| Score : 6206.381105921658:  80%|████████░░| 8/10 [00:01<00:00,  6.28it/s]
SwarmPy:   0%|░░░░░░░░░░| 0/1 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
simple_score = []
for _ in range(5):
    G = deepcopy(G1)
    aco_final = ACO_Pipeline(
        [
            ("Planner", Planner({'alpha' : 1., 'beta' : 6., 'q'  : 0})),
            ("Sol", SolutionConstructor()),
            ("DA", DaemonActions()),
            ("Updater", BestTourPheromonesUpdater(bounds=[0.1,1.])),
            
    ], iter_max=50, verbose=50
    )
    scores_simple = aco_final.run(G)
    scores_simple = np.array([el[1] for el in scores_simple['solutions']]) - opt_score

    simple_score.append(scores_simple[20:])

    

SwarmPy | Score : 7658.958320375771: 100%|██████████| 50/50 [00:10<00:00,  4.57it/s]
SwarmPy | Score : 7544.36590190409: 100%|██████████| 50/50 [00:10<00:00,  4.62it/s] 
SwarmPy | Score : 7658.958320375772: 100%|██████████| 50/50 [00:11<00:00,  4.44it/s]
SwarmPy | Score : 7679.152496997891: 100%|██████████| 50/50 [00:11<00:00,  4.35it/s] 
SwarmPy | Score : 7544.365901904089: 100%|██████████| 50/50 [00:11<00:00,  4.52it/s]


In [ ]:
import pandas as pd

In [ ]:
px.scatter(pd.DataFrame({'v1' : np.mean(scores, axis=0), 'v2' : np.mean(simple_score, axis=0)}),y=['v1', 'v2'], x = np.arange(30))

# plt.plot(scores_w_daemon)

# plt.xlabel('Iterations')
# plt.ylabel('Gap between best solution and optimal solution')
# plt.show()